# Выбор локации для скважины

**Цель проекта** - выбор локации для новой скважины на основе данных из трёх регионов.


Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

`id` — уникальный идентификатор скважины;<br>
`f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);<br>
`product` — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd                     # импорт библиотек
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score 
import sweetviz as sv
import warnings
warnings.filterwarnings('ignore')
from scipy import stats as st

In [2]:
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv') # чтение файла с данными и сохранение в df0
except:
    df0 = pd.read_csv('C:\\Users\\User\\Downloads\\geo_data_0.csv')

In [3]:
try:
    df1 = pd.read_csv('/datasets/geo_data_1.csv') # чтение файла с данными и сохранение в df1
except:
    df1 = pd.read_csv('C:\\Users\\User\\Downloads\\geo_data_1.csv')

In [4]:
try:
    df2 = pd.read_csv('/datasets/geo_data_2.csv') # чтение файла с данными и сохранение в df2
except:
    df2 = pd.read_csv('C:\\Users\\User\\Downloads\\geo_data_2.csv')

In [5]:
df0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [6]:
df1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [7]:
df2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [8]:
df0.drop(['id'], axis = 1, inplace = True) # удаление колонки id в df0

In [9]:
df1.drop(['id'], axis = 1, inplace = True) # удаление колонки id в df1

In [10]:
df2.drop(['id'], axis = 1, inplace = True) # удаление колонки id в df2 

In [11]:
report0 = sv.analyze([df0 ,'DF0'])
report0.show_html('common analysis.html')

In [12]:
report1 = sv.analyze([df1 ,'DF1'])
report1.show_html('common analysis.html')

In [13]:
report2 = sv.analyze([df2 ,'DF2'])
report2.show_html('common analysis.html')

- Пропусков и дубликатов в данных нет. 
- Колонка id удалена, так как не нужна для обучения модели.

## Обучение и проверка модели

#### Модель для первого региона

In [14]:
df0_train, df0_valid = train_test_split(df0, test_size=0.25, random_state=12345)

features_train = df0_train.drop(['product'], axis=1)     # признаки для обучающей выборки                        
target_train = df0_train['product']                      # целевой признак для обучающей выборки   

features_valid = df0_valid.drop(['product'], axis=1)     # признаки для валидационной выборки 
target_valid = df0_valid['product']                      # целевой признак для валидационной выборки 

In [15]:
def mean_well (features_train, target_train, features_valid, target_valid):
    model = LinearRegression().fit(features_train, target_train)
    predictions = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predictions)**0.5
    mean = predictions.mean()
    return print("Средний запас предсказанного сырья:",mean, "RMSE", rmse)

In [16]:
model_df0 = LinearRegression().fit(features_train, target_train)  
predictions_df0 = model_df0.predict(features_valid) # сохранение predictions_df0 для будущих расчетов

In [17]:
mean_well(features_train, target_train, features_valid, target_valid)

Средний запас предсказанного сырья: 92.59256778438035 RMSE 37.5794217150813


In [18]:
df0['product'].mean()

92.49999999999974

#### Модель для второго региона

In [19]:
df1_train, df1_valid = train_test_split(df1, test_size=0.25, random_state=12345)

features_train_df1 = df1_train.drop(['product'], axis=1)     # признаки для обучающей выборки                        
target_train_df1 = df1_train['product']                      # целевой признак для обучающей выборки   

features_valid_df1 = df1_valid.drop(['product'], axis=1)     # признаки для валидационной выборки 
target_valid_df1 = df1_valid['product']                      # целевой признак для валидационной выборки 

In [20]:
model_df1 = LinearRegression().fit(features_train_df1, target_train_df1)
predictions_df1 = model_df1.predict(features_valid_df1) # сохранение predictions_df1 для будущих расчетов

In [21]:
mean_well(features_train_df1, target_train_df1, features_valid_df1, target_valid_df1)

Средний запас предсказанного сырья: 68.72854689544602 RMSE 0.893099286775617


In [22]:
df1['product'].mean() # средний запас сырья в регионе 2

68.82500000002561

#### Модель для третьего региона

In [23]:
df2_train, df2_valid = train_test_split(df2, test_size=0.25, random_state=12345)

features_train_df2 = df2_train.drop(['product'], axis=1)     # признаки для обучающей выборки                        
target_train_df2 = df2_train['product']                      # целевой признак для обучающей выборки   

features_valid_df2 = df2_valid.drop(['product'], axis=1)     # признаки для валидационной выборки 
target_valid_df2 = df2_valid['product']                      # целевой признак для валидационной выборки 

In [24]:
model_df2 = LinearRegression().fit(features_train_df2, target_train_df2)
predictions_df2 = model_df2.predict(features_valid_df2) # сохранение predictions_df2 для будущих расчетов

In [25]:
mean_well(features_train_df2, target_train_df2, features_valid_df2, target_valid_df2)

Средний запас предсказанного сырья: 94.96504596800489 RMSE 40.02970873393434


In [26]:
df2['product'].mean()  # средний запас сырья в регионе 3

95.00000000000041

| Регионы    |  Средний запас сырья | Средний запас предсказанного сырья  |  RMSE | 
|:------------- |:---------------:| :------------- |:---------------:|
| Регион 1     | 92.5                            | 92.59                     | 37.57        | 
| Регион 2     | 68                           | 68.72                     | 0.89        | 
| Регион 3     | 95                           | 94.96                    | 40.02        | 

- В регионе 1 средний запас предсказанного сырья: 92.59,  RMSE 37.57
- В регионе 2 средний запас предсказанного сырья: 68.72,  RMSE 0.89
- В регионе 3 средний запас предсказанного сырья: 94.96,  RMSE 40.02 <br>
В регионах 1 и 3 средний запас предсказанного сырья достаточно высокий, но RMSE так же достаточно высок. Модель плохо предсказывает запас сырья. Для региона 2 модель лучше предсказывает запасы сырья, но средний запас предсказаного сырья в регионе 2 намного меньше, чем в регионах 1 и 3.

## Подготовка к расчёту прибыли

In [27]:
BUDGET = 10 ** 10   # сохранение значений в отдельные переменные
INCOME = 450000
WELLS = 500
SELECTED_WELLS = 200

In [28]:
breakeven_point = BUDGET / (INCOME * 200)

In [29]:
breakeven_point # минимальный объем баррелей нефти для безубыточной разработки

111.11111111111111

In [30]:
print("Средний запас в первом регионе", df0['product'].mean())
print("Средний запас во втором регионе", df1['product'].mean())
print("Средний запас в третьем регионе", df2['product'].mean())

Средний запас в первом регионе 92.49999999999974
Средний запас во втором регионе 68.82500000002561
Средний запас в третьем регионе 95.00000000000041


Средний запас во всех регионах меньше, чем минимальный запас необходимый для безубыточной разработки.

## Расчёт прибыли и рисков 

In [31]:
target_0 = pd.Series(target_valid).reset_index(drop=True)
target_1 = pd.Series(target_valid_df1).reset_index(drop=True)
target_2 = pd.Series(target_valid_df2).reset_index(drop=True)

In [32]:
predict_0 = pd.Series(*predictions_df0.reshape(1,-1))
predict_1 = pd.Series(*predictions_df1.reshape(1,-1))
predict_2 = pd.Series(*predictions_df2.reshape(1,-1))

In [33]:
def revenue(target, predictions, selected_count):                    # функция для расчета прибыли
    predictions_sorted = predictions.sort_values(ascending=False)
    selected = target.loc[predictions_sorted.index][:selected_count]
    
    result = selected.sum() * INCOME - BUDGET
    
    return result

In [34]:
revenue(target_0, predict_0, SELECTED_WELLS) # прибыль в регионе 1

3320826043.1398506

In [35]:
revenue(target_1, predict_1, SELECTED_WELLS) # прибыль в регионе 2

2415086696.681511

In [36]:
revenue(target_2, predict_2, SELECTED_WELLS) # прибыль в регионе 3

2710349963.5998325

In [37]:
state = np.random.RandomState(12345)
def risks (target, predict, WELLS, SELECTED_WELLS):

    values = []
    for i in range(1000):
        target_subsample = target.sample(n=WELLS, replace=True, random_state=state)
        probs_subsample = predict[target_subsample.index]

        values.append(revenue(target_subsample, probs_subsample, SELECTED_WELLS))

    values = pd.Series(values)

    mean = values.mean()
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    loss_count = 0
    for i in values :
        if i < 0 :
            loss_count += 1
    loss_rate = loss_count / 1000 * 100  
     
    return print('Доверительный интервал, min: {:,.2f}'.format(lower),
                 'Доверительный интервал, max: {:,.2f}'.format(upper),
                 "Средняя прибыль: {:,.2f}".format(mean), "Риск убытков, %", loss_rate, sep="\n")

In [38]:
risks(target_0, predict_0, WELLS, SELECTED_WELLS)

Доверительный интервал, min: -102,090,094.84
Доверительный интервал, max: 947,976,353.36
Средняя прибыль: 425,938,526.91
Риск убытков, %
6.0


In [39]:
risks(target_1, predict_1, WELLS, SELECTED_WELLS)

Доверительный интервал, min: 128,123,231.43
Доверительный интервал, max: 953,612,982.07
Средняя прибыль: 518,259,493.70
Риск убытков, %
0.3


In [40]:
risks(target_2, predict_2, WELLS, SELECTED_WELLS)

Доверительный интервал, min: -115,852,609.16
Доверительный интервал, max: 989,629,939.84
Средняя прибыль: 420,194,005.34
Риск убытков, %
6.2


Самая высокая средняя прибыль 518 млн.руб. и самый низкий риск убытков 0,3% получается для региона 2. Рекомендую второй регион для разработки.